In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

import pandas as pd
import time

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [7]:
titles = []
urls = []
area = []
city = []
company = []
job_type = []

In [8]:
headers = {"User-Agent": "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.2.8) Gecko/20100722 Firefox/3.6.8 GTB7.1 (.NET CLR 3.5.30729)"}
base_url = "https://wuzzuf.net/search/jobs/?a=navbg&filters%5Bcountry%5D%5B0%5D=Egypt&q=Data-Analyst&start=0"

response = requests.get(base_url, headers = headers, timeout = 5)

html = response.content
soup = BeautifulSoup(html, "lxml")

page_end = (int(soup.find("strong").text) // 15) + 1 

In [9]:
for page in range(0, page_end):
    
    base_url = f"https://wuzzuf.net/search/jobs/?a=navbg&filters%5Bcountry%5D%5B0%5D=Egypt&q=Data-Analyst&start={page}"
    response = requests.get(base_url, headers=headers)
    
    if response.status_code == 200:            
        print('URL #{0}: {1}'.format(page+1, base_url))
    else:                                  
        print('Status code {0}: Skipping URL #{1}: {2}'.format(response.status_code, page+1, url))
        continue    
        
    html = response.content
    soup = BeautifulSoup(html, "lxml")
    
    #--------------------------------------------------------------------------------#
    
    anchors = soup.find_all('a', {"rel": "noreferrer", "class": "css-o171kl"})
    
    title = [title.string for title in anchors]
    titles.extend(title)
    
    absolute_url = [urljoin(base_url, link.get('href')) for link in anchors]
    urls.extend(absolute_url)
    
    #--------------------------------------------------------------------------------#
    
    location = soup.find_all('span', class_ = 'css-5wys0k')
    
    areas = [loc.text.split(',')[0].strip() for loc in location]
    area.extend(areas)
    
    cities = [loc.text.split(',')[1].strip() for loc in location]
    city.extend(cities)
    
    #--------------------------------------------------------------------------------#
    
    company_soup = soup.find_all('a', {"rel": "noreferrer", "class": "css-17s97q8"})
    
    companies = [comp.string.strip(' -') for comp in company_soup]
    company.extend(companies)
    
    #--------------------------------------------------------------------------------#
    
    job_types = soup.find_all('div', class_ = 'css-1lh32fc')
    
    for job in job_types:
        job_type.append(" | ".join([type.text.strip(" .") for type in job.find_all('span')]))
    
    time.sleep(3)

URL #1: https://wuzzuf.net/search/jobs/?a=navbg&filters%5Bcountry%5D%5B0%5D=Egypt&q=Data-Analyst&start=0
URL #2: https://wuzzuf.net/search/jobs/?a=navbg&filters%5Bcountry%5D%5B0%5D=Egypt&q=Data-Analyst&start=1
URL #3: https://wuzzuf.net/search/jobs/?a=navbg&filters%5Bcountry%5D%5B0%5D=Egypt&q=Data-Analyst&start=2
URL #4: https://wuzzuf.net/search/jobs/?a=navbg&filters%5Bcountry%5D%5B0%5D=Egypt&q=Data-Analyst&start=3
URL #5: https://wuzzuf.net/search/jobs/?a=navbg&filters%5Bcountry%5D%5B0%5D=Egypt&q=Data-Analyst&start=4
URL #6: https://wuzzuf.net/search/jobs/?a=navbg&filters%5Bcountry%5D%5B0%5D=Egypt&q=Data-Analyst&start=5
URL #7: https://wuzzuf.net/search/jobs/?a=navbg&filters%5Bcountry%5D%5B0%5D=Egypt&q=Data-Analyst&start=6
URL #8: https://wuzzuf.net/search/jobs/?a=navbg&filters%5Bcountry%5D%5B0%5D=Egypt&q=Data-Analyst&start=7
URL #9: https://wuzzuf.net/search/jobs/?a=navbg&filters%5Bcountry%5D%5B0%5D=Egypt&q=Data-Analyst&start=8
URL #10: https://wuzzuf.net/search/jobs/?a=navbg&filter

In [10]:
experience_needed = []
career_level = []
education_level = []
salary = []
job_categories = []
skills_and_tools = []
job_description = []
job_requirements = []

In [11]:
options = Options()
options.add_argument('--disable-blink-features=AutomationControlled')

Path = Service("C:\webdrivers\chromedriver.exe")

In [12]:
for url in urls:
        
    driver = webdriver.Chrome(service = Path, options = options)
    driver.get(url)
    
    time.sleep(3)
    
    try:
        main = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.TAG_NAME, "main"))
        )
    
        spans = main.find_elements(By.CLASS_NAME, 'css-4xky9y')

        experience_needed.append(spans[0].text)
        career_level.append(spans[1].text)
        education_level.append(spans[2].text)
        salary.append(spans[-1].text)
    
        #--------------------------------------------------------------------------------#

        categories = main.find_elements(By.XPATH, '//div[@class="css-13sf2ik"]/ul/li/a/span')
        job_categories_ = " | ".join([category.text for category in categories])
        job_categories.append(job_categories_)

        tools = main.find_elements(By.XPATH, '//div[@class="css-s2o0yh"]/a/span/span/span')
        skills_and_tools_ = " | ".join([tool.text for tool in tools])
        skills_and_tools.append(skills_and_tools_)

        descriptions = main.find_elements(By.XPATH, '//div[@class="css-1uobp1k"]/ul/li')
        job_description_ = " | ".join([description.text.strip(" .") for description in descriptions])
        job_description.append(job_description_)

        requirements = main.find_elements(By.XPATH, '//div[@class="css-1t5f0fr"]/ul/li')
        job_requirements_ = " | ".join([requirement.text.strip(" .") for requirement in requirements])
        job_requirements.append(job_requirements_)
    
    except:
        experience_needed.append(None)
        career_level.append(None)
        education_level.append(None)
        salary.append(None)
        job_categories.append(None)
        skills_and_tools.append(None)
        job_description.append(None)
        job_requirements.append(None)
        
    finally:
        driver.quit()
        time.sleep(3)

In [14]:
dic_of_lists = {'titles': titles, 'area': area, 
                'city': city, 'company': company, 
                'job_type': job_type, 'experience_needed': experience_needed,
                'career_level': career_level, 'education_level': education_level,
                'salary': salary, 'job_categories': job_categories,
                'skills_and_tools': skills_and_tools, 'urls': urls,
                'job_description': job_description, 'job_requirements': job_requirements}

data = pd.DataFrame(dic_of_lists)

data.to_csv('WUZZUF_WebScarping.csv')

In [23]:
data.head()

,titles,area,city,company,job_type,experience_needed,career_level,education_level,salary,job_categories,skills_and_tools,urls,job_description,job_requirements
0,Mid Senior Data Analyst,Maadi,Cairo,Worex,Full Time,More Than 3 Years,Experienced (Non-Manager),Bachelor'S Degree,Confidential,IT/Software Development | Analyst/Research | E...,Data Analysis | Data Analyst | Information Tec...,https://wuzzuf.net/jobs/p/0T6a4KDMsFNn-Mid-Sen...,"Track, collect, and interpret data, then analy...",3+ years in a data analyst role at least | Per...
1,Junior Data Analyst,Maadi,Cairo,Worex,Full Time,1 To 2 Years,Entry Level (Junior Level / Fresh Grad),Bachelor'S Degree,Confidential,IT/Software Development | Analyst/Research | E...,Analyst | Data Analyst | Data Analysis | Infor...,https://wuzzuf.net/jobs/p/iq5jLz08Y7Fb-Junior-...,"Deliver dashboards, analytics & Reports migrat...",Demonstrated experience as a Data Analyst in a...
2,Senior Data Analyst - Business Intelligence,Maadi,Cairo,Confidential,Full Time,3 To 5 Years,Experienced (Non-Manager),Bachelor'S Degree,10000 To 12000 EGP Per Month,IT/Software Development | Analyst/Research | E...,Analyst | Business Administration | business |...,https://wuzzuf.net/jobs/p/FdG6WamryKOU-Senior-...,"Track, collect, and interpret data, then analy...",3+ years in a data analyst role at least | Per...
3,E-Commerce & Marketing Data Analyst,New Cairo,Cairo,Confidential,Full Time,3 To 6 Years,Experienced (Non-Manager),Bachelor'S Degree,Confidential,Marketing/PR/Advertising | Analyst/Research | ...,Market Research | Data Analysis | Analysis | M...,https://wuzzuf.net/jobs/p/jzNIjX873nvy-E-Comme...,Reviewing and collecting information from seco...,3-6 years of Experience
4,Data Analyst,New Cairo,Cairo,Arma Foods,Full Time,3 To 5 Years,Experienced (Non-Manager),Bachelor'S Degree,Confidential,Analyst/Research,Data Analysis | Analysis | Computer Science | ...,https://wuzzuf.net/jobs/p/3ZTNoLhqBc6T-Data-An...,Develop the data analytics methods to collect ...,"Degree in Business, Technology, Engineering, o..."
